In [149]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import json
import pickle as pkl
import re

In [150]:
cwd_path = os.getcwd()
cwd_path

'/Users/arjbingly/Documents/GitHub/DataDrivenHealers_HumanaMaysHealthcareAnalytics23'

### Codes related to Side Effects of Osimertinib

The ICD-10 codes related to the side effects have been extracted in the file 'side_effect_codes.json' (as a dict)

In [151]:
with open(f'{cwd_path}/SideEffects_ICD10/side_effect_codes.json', 'r') as f:
    sf_codes = json.load(f)

Side effect groups

In [152]:
sf_codes.keys()

dict_keys(['Diarrhea', 'Eye', 'Infection', 'Nail', 'Neutropenia', 'QT', 'Skin', 'Stevens-Jhonson', 'Stomatitis', 'Thrombocytopenia'])

In [173]:
len(sf_codes.keys())

9

In [153]:
print(f'No of codes:')
for key in sf_codes.keys():
    print(f'{key} : {len(sf_codes[key])}')

No of codes:
Diarrhea : 23
Eye : 2531
Infection : 499
Nail : 1227
Neutropenia : 5
QT : 1
Skin : 468
Stevens-Jhonson : 0
Stomatitis : 6
Thrombocytopenia : 38


In [154]:
sf_codes.pop('Stevens-Jhonson') #Since no codes exist

[]

### Medical Claims

In [155]:
medclms = pd.read_pickle(f'{cwd_path}/medclms_train_new.pkl')

In [156]:
medclms.head()

,therapy_id,medclm_key,clm_unique_key,primary_diag_cd,visit_date,diag_cd2,diag_cd3,diag_cd4,diag_cd5,diag_cd6,...,ade_diagnosis,seizure_diagnosis,pain_diagnosis,fatigue_diagnosis,nausea_diagnosis,hyperglycemia_diagnosis,constipation_diagnosis,diarrhea_diagnosis,primary_diag_parent_cd,primary_diag_group_cd
0,1066310426-TAGRISSO-1,35908472910,649143000473431803,I70292,2019-09-10T00:00:00.000Z,I771,I739,R9431,Z7982,Z79899,...,0,0,0,0,0,0,0,0,I70,I00-I99
1,1094522162-TAGRISSO-1,37189420103,934834000476811903,J9601,2020-08-21T00:00:00.000Z,J910,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,J96,J00-J99
2,1104741185-TAGRISSO-1,37128499255,748244100476652103,E785,2020-07-14T00:00:00.000Z,I10,J324,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,E78,E00-E89
3,1012749208-TAGRISSO-1,36269844999,589034000474672103,C3492,2020-02-04T00:00:00.000Z,I825Z9,T83518A,N390,J942,J910,...,0,0,0,0,0,0,0,0,C34,C00-D49
4,1000110916-TAGRISSO-1,36448002258,663113000474981803,C3490,2020-03-05T00:00:00.000Z,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,C34,C00-D49


In [157]:
medclms.columns

Index(['therapy_id', 'medclm_key', 'clm_unique_key', 'primary_diag_cd',
       'visit_date', 'diag_cd2', 'diag_cd3', 'diag_cd4', 'diag_cd5',
       'diag_cd6', 'diag_cd7', 'diag_cd8', 'diag_cd9', 'process_date',
       'reversal_ind', 'pot', 'util_cat', 'hedis_pot', 'clm_type',
       'ade_diagnosis', 'seizure_diagnosis', 'pain_diagnosis',
       'fatigue_diagnosis', 'nausea_diagnosis', 'hyperglycemia_diagnosis',
       'constipation_diagnosis', 'diarrhea_diagnosis',
       'primary_diag_parent_cd', 'primary_diag_group_cd'],
      dtype='object')

The medclms dataframe has a person identifier - 'therapy_id'
primary diagnosis code - 'primary_diag_cd'
secondary diagnosis codes - 'diag_cd2', 'daig_cd3', ... ,'diag_cd9'

In [158]:
# cols with diagnosis codes
diag_cols = ['primary_diag_cd', 'diag_cd2', 'diag_cd3', 'diag_cd4', 'diag_cd5', 'diag_cd6', 'diag_cd7', 'diag_cd8', 'diag_cd9']

In [159]:
# makign the person_id as index
medclms = medclms.set_index('therapy_id')

## Eg. Side Effect relating Eye

In [160]:
sf_codes['Eye']

['H2521',
 'H02209',
 'C6991',
 'H44012',
 'H02869',
 'H16331',
 'H4301',
 'H16139',
 'T1580XS',
 'H11112',
 'H35441',
 'H15039',
 'H16142',
 'H40212',
 'T1590XD',
 'E133521',
 'H59019',
 'H00015',
 'D23122',
 'E133552',
 'T1511XD',
 'H33111',
 'H35359',
 'H5440',
 'T1511XS',
 'T2621XS',
 'H2011',
 'H02823',
 'T1502XS',
 'H16119',
 'H4053X4',
 'H4610',
 'H11232',
 'H3322',
 'H18022',
 'H442D9',
 'H16031',
 'H348110',
 'H59362',
 'H26129',
 'T1581XA',
 'T868411',
 'H01029',
 'C43122',
 'H25041',
 'H4901',
 'H40811',
 'H16291',
 'S00242S',
 'H11032',
 'H4052X3',
 'S01141A',
 'S01142S',
 'H40152',
 'H44812',
 'H35362',
 'H4031X0',
 'E093212',
 'H44441',
 'B0239',
 'H18002',
 'C441291',
 'H35461',
 'E093591',
 'H11132',
 'H401114',
 'H4031X2',
 'H59022',
 'H16432',
 'H02856',
 'H15019',
 'H16011',
 'H353293',
 'S01151S',
 'H59031',
 'H26002',
 'D03111',
 'H10531',
 'T2652XD',
 'H02146',
 'H59212',
 'S00222S',
 'H02886\n',
 'H05241',
 'R402132',
 'H18029',
 'H18451',
 'H47031',
 'H16232',
 

There are 2 ways to count eye (the side effect considered) related codes per person
1. Count the number of codes related to the side effect in a person's medical record. (hereafter called 'countall')
2. Count the number of unique codes related to the side effect in a person's medical record. (hereafter called 'countuniq')

### 1. Countall

In [163]:
# eg.
## for primary_diag
### for eye
countall = medclms[medclms.primary_diag_cd.isin(sf_codes['Eye'])].groupby('therapy_id').primary_diag_cd.count().reset_index()
countall.columns = ['therapy_id', 'eye_pd_count_all']
countall

,therapy_id,eye_pd_count_all
0,1000867560-TAGRISSO-1,5
1,1004400309-TAGRISSO-1,18
2,1005182640-TAGRISSO-1,8
3,1005693365-TAGRISSO-1,4
4,1005964858-TAGRISSO-1,3
...,...,...
70,1116339224-TAGRISSO-1,1
71,1118113212-TAGRISSO-1,7
72,1118371981-TAGRISSO-1,2
73,1118630665-TAGRISSO-1,3


#### For all side effects

In [167]:
## for primary_diag
### All side effects
df_list = []
for side_effect in sf_codes.keys():
    countall = medclms[medclms.primary_diag_cd.isin(sf_codes[side_effect])].groupby('therapy_id').primary_diag_cd.count().reset_index()
    countall.columns = ['therapy_id', f'{side_effect.lower()}_pd_count_all']
    countall = countall.set_index('therapy_id')
    df_list.append(countall)
pd_count_all_df = df_list[0].join(df_list[1:], how='outer')

In [168]:
pd_count_all_df

,diarrhea_pd_count_all,eye_pd_count_all,infection_pd_count_all,nail_pd_count_all,neutropenia_pd_count_all,qt_pd_count_all,skin_pd_count_all,stomatitis_pd_count_all,thrombocytopenia_pd_count_all
therapy_id,,,,,,,,,
1008520415-TAGRISSO-1,1.0,46.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1011089073-TAGRISSO-1,3.0,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN
1013219968-TAGRISSO-1,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1016103876-TAGRISSO-1,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1016891647-TAGRISSO-1,17.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
1013495938-TAGRISSO-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1016166941-TAGRISSO-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
1054868180-TAGRISSO-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0


In [169]:
## for all sec diag
### All side effects
df_list = []
for side_effect in sf_codes.keys():
    df_list_ = []
    for diag_col in diag_cols[1:]: 
        countall = medclms[medclms[diag_col].isin(sf_codes[side_effect])].groupby('therapy_id')[diag_col].count().reset_index()
        countall.columns = ['therapy_id', f'{side_effect.lower()}_{diag_col}_count_all']
        countall = countall.set_index('therapy_id')
        df_list_.append(countall)
    df_ = pd.DataFrame(df_list_[0].join(df_list_[1:], how='outer').sum(axis=1)).reset_index()
    df_.columns =['therapy_id', f'{side_effect.lower()}_sd_count_all']
    df_ = df_.set_index('therapy_id')
    df_list.append(df_)
sd_count_all_df = df_list[0].join(df_list[1:], how='outer')

In [170]:
sd_count_all_df

,diarrhea_sd_count_all,eye_sd_count_all,infection_sd_count_all,nail_sd_count_all,neutropenia_sd_count_all,qt_sd_count_all,skin_sd_count_all,stomatitis_sd_count_all,thrombocytopenia_sd_count_all
therapy_id,,,,,,,,,
1010187115-TAGRISSO-1,2.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1010393275-TAGRISSO-1,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1013219968-TAGRISSO-1,5.0,NaN,20.0,NaN,NaN,NaN,NaN,NaN,NaN
1016783257-TAGRISSO-1,8.0,NaN,NaN,NaN,7.0,NaN,13.0,4.0,15.0
1027867104-TAGRISSO-1,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
1107106465-TAGRISSO-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1122173495-TAGRISSO-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1107373430-TAGRISSO-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0


In [175]:
count_all_df = pd_count_all_df.join(sd_count_all_df, how='outer')
print(count_all_df.columns)
print(f'No of columns : {len(count_all_df.columns)}')
print(f'No of rows    : {len(count_all_df)}')

Index(['diarrhea_pd_count_all', 'eye_pd_count_all', 'infection_pd_count_all',
       'nail_pd_count_all', 'neutropenia_pd_count_all', 'qt_pd_count_all',
       'skin_pd_count_all', 'stomatitis_pd_count_all',
       'thrombocytopenia_pd_count_all', 'diarrhea_sd_count_all',
       'eye_sd_count_all', 'infection_sd_count_all', 'nail_sd_count_all',
       'neutropenia_sd_count_all', 'qt_sd_count_all', 'skin_sd_count_all',
       'stomatitis_sd_count_all', 'thrombocytopenia_sd_count_all'],
      dtype='object')
No of columns : 18
No of rows    : 364


### 2. Countuniq

In [178]:
# eg.
## for primary_diag
### for eye
countuniq = medclms[medclms.primary_diag_cd.isin(sf_codes['Eye'])].groupby('therapy_id').primary_diag_cd.nunique().reset_index()
countuniq.columns = ['therapy_id', 'eye_pd_count_unique']
countuniq

,therapy_id,eye_pd_count_unique
0,1000867560-TAGRISSO-1,1
1,1004400309-TAGRISSO-1,4
2,1005182640-TAGRISSO-1,2
3,1005693365-TAGRISSO-1,1
4,1005964858-TAGRISSO-1,1
...,...,...
70,1116339224-TAGRISSO-1,1
71,1118113212-TAGRISSO-1,2
72,1118371981-TAGRISSO-1,1
73,1118630665-TAGRISSO-1,2


#### For all side effects

In [180]:
## for primary_diag
### All side effects
df_list = []
for side_effect in sf_codes.keys():
    countall = medclms[medclms.primary_diag_cd.isin(sf_codes[side_effect])].groupby('therapy_id').primary_diag_cd.nunique().reset_index()
    countall.columns = ['therapy_id', f'{side_effect.lower()}_pd_count_unique']
    countall = countall.set_index('therapy_id')
    df_list.append(countall)
pd_count_unique_df  = df_list[0].join(df_list[1:], how='outer')

In [181]:
pd_count_unique_df

,diarrhea_pd_count_unique,eye_pd_count_unique,infection_pd_count_unique,nail_pd_count_unique,neutropenia_pd_count_unique,qt_pd_count_unique,skin_pd_count_unique,stomatitis_pd_count_unique,thrombocytopenia_pd_count_unique
therapy_id,,,,,,,,,
1008520415-TAGRISSO-1,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1011089073-TAGRISSO-1,2.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
1013219968-TAGRISSO-1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1016103876-TAGRISSO-1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1016891647-TAGRISSO-1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
1013495938-TAGRISSO-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1016166941-TAGRISSO-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1054868180-TAGRISSO-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [182]:
## for all sec diag
### All side effects
df_list = []
for side_effect in sf_codes.keys():
    df_list_ = []
    for diag_col in diag_cols[1:]:
        countuniq = medclms[medclms[diag_col].isin(sf_codes[side_effect])].groupby('therapy_id')[diag_col].nunique().reset_index()
        countuniq.columns = ['therapy_id', f'{side_effect.lower()}_{diag_col}_count_unique']
        countuniq = countuniq.set_index('therapy_id')
        df_list_.append(countuniq)
    df_ = pd.DataFrame(df_list_[0].join(df_list_[1:], how='outer').sum(axis=1)).reset_index()
    df_.columns =['therapy_id', f'{side_effect.lower()}_sd_count_unique']
    df_ = df_.set_index('therapy_id')
    df_list.append(df_)
sd_count_unique_df = df_list[0].join(df_list[1:], how='outer')

In [183]:
sd_count_unique_df

,diarrhea_sd_count_unique,eye_sd_count_unique,infection_sd_count_unique,nail_sd_count_unique,neutropenia_sd_count_unique,qt_sd_count_unique,skin_sd_count_unique,stomatitis_sd_count_unique,thrombocytopenia_sd_count_unique
therapy_id,,,,,,,,,
1010187115-TAGRISSO-1,2.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1010393275-TAGRISSO-1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1013219968-TAGRISSO-1,1.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN
1016783257-TAGRISSO-1,4.0,NaN,NaN,NaN,2.0,NaN,4.0,2.0,2.0
1027867104-TAGRISSO-1,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
1107106465-TAGRISSO-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1122173495-TAGRISSO-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1107373430-TAGRISSO-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0


In [184]:
count_unique_df = pd_count_unique_df.join(sd_count_unique_df, how='outer')
print(count_unique_df.columns)
print(f'No of columns : {len(count_unique_df.columns)}')
print(f'No of rows    : {len(count_unique_df)}')

Index(['diarrhea_pd_count_unique', 'eye_pd_count_unique',
       'infection_pd_count_unique', 'nail_pd_count_unique',
       'neutropenia_pd_count_unique', 'qt_pd_count_unique',
       'skin_pd_count_unique', 'stomatitis_pd_count_unique',
       'thrombocytopenia_pd_count_unique', 'diarrhea_sd_count_unique',
       'eye_sd_count_unique', 'infection_sd_count_unique',
       'nail_sd_count_unique', 'neutropenia_sd_count_unique',
       'qt_sd_count_unique', 'skin_sd_count_unique',
       'stomatitis_sd_count_unique', 'thrombocytopenia_sd_count_unique'],
      dtype='object')
No of columns : 18
No of rows    : 364


## Final side effect features df

In [188]:
sf_df = count_all_df.join(count_unique_df, how='outer')
sf_df.to_pickle(f'{os.getcwd()}/side_effects_df.pkl')